In [2]:
import pandas as pd

In [3]:


## Read the data in data frame
df=pd.read_csv("GasBuddy_brand_prices.csv",delimiter='|')

df.head() ## Read top few values of the rows to ensure that data is read properly

,price,fuel_type,address,city,state,brand,date
0,2.99,Regular,6109 W William Cannon Dr,Austin,TX,Exxon,2018-07-01 00:00:00
1,3.29,Midgrade,6109 W William Cannon Dr,Austin,TX,Exxon,2018-07-01 00:00:00
2,3.59,Premium,6109 W William Cannon Dr,Austin,TX,Exxon,2018-07-01 00:00:00
3,2.89,Diesel,2440 19th St,Lubbock,TX,Phillips 66,2018-07-01 00:00:02
4,2.59,Regular,2440 19th St,Lubbock,TX,Phillips 66,2018-07-01 00:00:02


In [4]:
df.info()    ## Check first level information of the data frame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109350 entries, 0 to 109349
Data columns (total 7 columns):
price        109350 non-null float64
fuel_type    109350 non-null object
address      109350 non-null object
city         109350 non-null object
state        109350 non-null object
brand        109350 non-null object
date         109350 non-null object
dtypes: float64(1), object(6)
memory usage: 3.3+ MB


In [5]:
##### . Many columns above are of object type. To be more precise in analysis convert them to their proper type

## Below are categorical variables
df["fuel_type"] = df["fuel_type"].astype('category')
df["city"] = df["city"].astype('category')
df["state"] = df["state"].astype('category')
df["brand"] = df["brand"].astype('category')

## Date is in datetimeformat
df["date"] = df["date"].astype('datetime64[ns]')

### There is no need to convert address variable. 

df.info()     ## Check the info again to verify correctness of data types



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109350 entries, 0 to 109349
Data columns (total 7 columns):
price        109350 non-null float64
fuel_type    109350 non-null category
address      109350 non-null object
city         109350 non-null category
state        109350 non-null category
brand        109350 non-null category
date         109350 non-null datetime64[ns]
dtypes: category(4), datetime64[ns](1), float64(1), object(1)
memory usage: 2.6+ MB


In [6]:
## Check the summary of the data types

df.describe()  ## here only series variables will be summarized

,price
count,109350.000000
mean,2.853092
std,0.248598
min,2.190000
25%,2.650000
50%,2.850000
75%,2.990000
max,4.090000


Minimum value is 2.19 and max is 4.09
It means there is no missing or zero value. Also looking at the summary above there does not seem to be any outlier

In [7]:
df.describe(include = 'category')   ### Check the summary of categorical variables

,fuel_type,city,state,brand
count,109350,109350,109350,109350
unique,4,840,1,10
top,Regular,Houston,TX,Exxon
freq,42580,8121,109350,23439


In [8]:
df["brand"].value_counts()   ## We have to work on brand therefore focusing on the detailed summary of brand only

Exxon          23439
Valero         18915
7-Eleven       17161
Shell          17015
Chevron        12145
Texaco          5900
Conoco          5209
Alon            5029
Phillips 66     3244
CITGO           1293
Name: brand, dtype: int64

Output in above command shows that there is no missing value. Also there is no mis-spelled/duplicate category

In [9]:
df.duplicated().any()  #Check for duplicate rows
df[df.duplicated()].head()       #displaying all duplicates rows

df[df.duplicated()].shape


(72, 7)

There are duplicate rows in data.

Question : Are these duplicate rows valid ? Should they be considered in further analysis ? 

(For now removing duplicate records from analysis)

In [10]:
print("Number of rows in primary dataset - " + str(df.shape[0]))
print("Number of duplicate rows in primary dataset - " + str(df[df.duplicated()].shape[0]))
df.drop_duplicates(inplace=True)

print("Number of rows after duplicate removal - " + str(df.shape[0]))


Number of rows in primary dataset - 109350
Number of duplicate rows in primary dataset - 72
Number of rows after duplicate removal - 109278


In [11]:
#Select all rows for working columns only
workdf = df.loc[ :, ["price","brand", "date"]]
workdf.head()

,price,brand,date
0,2.99,Exxon,2018-07-01 00:00:00
1,3.29,Exxon,2018-07-01 00:00:00
2,3.59,Exxon,2018-07-01 00:00:00
3,2.89,Phillips 66,2018-07-01 00:00:02
4,2.59,Phillips 66,2018-07-01 00:00:02


Data above has both date and time. To aggregate on date basis only date information should be kept and time should not be considered. Therefore extracting the period information from the column and creating a new column

In [12]:
## Extract the period inf
workdf["period"] = workdf["date"].dt.to_period('D')

## now change the period to category
workdf["period"] = workdf["period"].astype("category")
workdf.head()

,price,brand,date,period
0,2.99,Exxon,2018-07-01 00:00:00,2018-07-01
1,3.29,Exxon,2018-07-01 00:00:00,2018-07-01
2,3.59,Exxon,2018-07-01 00:00:00,2018-07-01
3,2.89,Phillips 66,2018-07-01 00:00:02,2018-07-01
4,2.59,Phillips 66,2018-07-01 00:00:02,2018-07-01


In [13]:
## Now group by data based on brand and date and compute the mean
summary_report = workdf.groupby(['brand','period'])[['price']].mean()

summary_report.head()

price
brand    period              
7-Eleven 2018-07-01  2.848512
         2018-07-02  2.838686
         2018-07-03  2.835150
Alon     2018-07-01  2.946786
         2018-07-02  2.937942

In [14]:
## Write report to CSV

summary_report.to_csv("daily-price-summary-by-brand.csv")